In [1]:
import numpy as np
import torch
torch.randn(1).cpu().numpy()  # warm-up

array([1.2762235], dtype=float32)

In [2]:
# Install PyTorch Geometric and TGAT dependencies
!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.0.1+cu118.html
!pip install torch-geometric
!pip install torch-geometric-temporal

Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 529.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 110.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 87.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 14.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lit: filename=lit-15.0.7-py3-none-any.whl size=89990 sha256=d44fb14378f4d9a4e746e24fb3cfd18b755e5c1b39637458f02b411910c7aac9
  Stored in directory: /root/.cache/pip/wheels/fc/5d/45/34fe9945d5e45e261134e72284395be36c2d4828af38e2b0fe
Successfully built lit
  Attempting uninstall: triton
    Found existing installation: triton 3.2.0
    Uninstalling triton-3.2.0:
      Successfully uninstalled triton-3.2.0
  Attempting uninstall: torch
    Found existing installation: torch 2.6

Looking in links: https://data.pyg.org/whl/torch-2.0.1+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 91.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 114.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 886.5/886.5 kB 58.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.9/98.9 kB 9.8 MB/s eta 0:00:00


In [1]:


import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import TransformerConv, global_mean_pool, BatchNorm, GATConv
from torch_geometric.data import Data, DataLoader, TemporalData
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import networkx as nx
from collections import defaultdict
import warnings
import math
warnings.filterwarnings('ignore')
from torch_geometric.nn import GATv2Conv as GATConv


Disabling PyTorch because PyTorch >= 2.1 is required but found 2.0.1+cu118

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in st

In [2]:
# -*- coding: utf-8 -*-
"""TGAT_CAN_IDS_100%_Accuracy.ipynb

TGAT (Temporal Graph Attention Network) implementation for CAN bus intrusion detection
Replicating the 99.99% accuracy achieved with GCN using temporal attention mechanisms
"""



class CANDataProcessor:
    def __init__(self):
        self.scaler = StandardScaler()
        self.label_encoder = LabelEncoder()

    def load_otids_data(self, file_paths):
        all_data = []

        for file_path, label in file_paths.items():
            print(f"Processing {file_path}...")

            try:
                data = []

                with open(file_path, 'r') as f:
                    for line in f:
                        line = line.strip()
                        if not line:
                            continue

                        parts = line.split()
                        timestamp = float(parts[1])
                        can_id = int(parts[3], 16)
                        dlc = int(parts[6])
                        data_bytes = [int(byte, 16) for byte in parts[7:7 + dlc]]

                        # Pad data to 8 bytes
                        while len(data_bytes) < 8:
                            data_bytes.append(0)

                        row = [timestamp, can_id, dlc] + data_bytes
                        data.append(row)

                df = pd.DataFrame(data, columns=[
                    'timestamp', 'can_id', 'dlc',
                    'data0', 'data1', 'data2', 'data3',
                    'data4', 'data5', 'data6', 'data7'
                ])
                df['label'] = label
                all_data.append(df)

                print(f"✔ Loaded {len(df)} rows from {file_path}")

            except Exception as e:
                print(f"❌ Error parsing {file_path}: {e}")

        if not all_data:
            print("⚠ No data loaded. Returning empty DataFrame.")
            return pd.DataFrame()

        return pd.concat(all_data, ignore_index=True)

    def create_sample_data(self):
        """Create sample CAN data for demonstration if files not found"""
        np.random.seed(42)
        sample_data = []

        # Generate sample data for each attack type
        for label in ['normal', 'dos', 'fuzzy', 'impersonation']:
            n_samples = 5000

            # CAN message features: timestamp, can_id, dlc, data0-data7
            if label == 'normal':
                # Normal traffic patterns
                timestamps = np.arange(n_samples) * 0.001  # Regular intervals
                can_ids = np.random.choice([0x123, 0x456, 0x789], n_samples)
                dlc = np.random.choice([8], n_samples)
                data = np.random.randint(0, 256, (n_samples, 8))
            elif label == 'dos':
                # DoS attack - high frequency, same ID
                timestamps = np.arange(n_samples) * 0.0001  # High frequency
                can_ids = np.full(n_samples, 0x123)
                dlc = np.random.choice([8], n_samples)
                data = np.random.randint(0, 256, (n_samples, 8))
            elif label == 'fuzzy':
                # Fuzzy attack - random data with irregular timestamps
                timestamps = np.sort(np.random.random(n_samples) * 10)
                can_ids = np.random.randint(0, 2048, n_samples)
                dlc = np.random.choice(range(1, 9), n_samples)
                data = np.random.randint(0, 256, (n_samples, 8))
            else:  # impersonation
                # Impersonation - legitimate ID with suspicious temporal patterns
                timestamps = np.arange(n_samples) * 0.005  # Irregular intervals
                can_ids = np.random.choice([0x123, 0x456], n_samples)
                dlc = np.random.choice([8], n_samples)
                data = np.random.randint(200, 256, (n_samples, 8))

            # Combine features
            features = np.column_stack([timestamps, can_ids, dlc, data])
            df = pd.DataFrame(features, columns=[
                'timestamp', 'can_id', 'dlc',
                'data0', 'data1', 'data2', 'data3',
                'data4', 'data5', 'data6', 'data7'
            ])
            df['label'] = label
            sample_data.append(df)

        print("Created sample dataset with 20,000 CAN messages")
        return pd.concat(sample_data, ignore_index=True)

    def preprocess_can_data(self, df):
        """Preprocess CAN bus data"""
        feature_cols = [col for col in df.columns if col != 'label']

        # Handle missing values
        df = df.fillna(0)

        # Extract features
        X = df[feature_cols].values
        y = df['label'].values

        # Normalize features
        X_scaled = self.scaler.fit_transform(X)

        # Encode labels
        y_encoded = self.label_encoder.fit_transform(y)

        return X_scaled, y_encoded, df['timestamp'].values

    def create_temporal_graph(self, X, y, timestamps, window_size=15):
        """Create temporal graph structure from CAN data for TGAT"""
        graphs = []

        # Sort by timestamp to ensure temporal order
        sort_idx = np.argsort(timestamps)
        X_sorted = X[sort_idx]
        y_sorted = y[sort_idx]
        timestamps_sorted = timestamps[sort_idx]

        for i in range(0, len(X_sorted) - window_size + 1, window_size//2):
            # Get window of data
            window_data = X_sorted[i:i+window_size]
            window_labels = y_sorted[i:i+window_size]
            window_timestamps = timestamps_sorted[i:i+window_size]

            # Create temporal adjacency matrix
            adj_matrix = np.zeros((window_size, window_size))
            time_matrix = np.zeros((window_size, window_size))

            # Temporal connections (consecutive messages)
            for j in range(window_size - 1):
                adj_matrix[j, j+1] = 1.0
                time_matrix[j, j+1] = window_timestamps[j+1] - window_timestamps[j]

            # Feature similarity connections with temporal decay
            for j in range(window_size):
                for k in range(j+1, window_size):
                    # Feature similarity
                    feature_sim = np.exp(-np.linalg.norm(window_data[j] - window_data[k]))

                    # Temporal decay
                    time_diff = abs(window_timestamps[k] - window_timestamps[j])
                    temporal_decay = np.exp(-time_diff / np.std(window_timestamps))

                    # Combined similarity
                    similarity = feature_sim * temporal_decay

                    if similarity > 0.3:  # Threshold for connection
                        adj_matrix[j, k] = similarity
                        adj_matrix[k, j] = similarity
                        time_matrix[j, k] = time_diff
                        time_matrix[k, j] = time_diff

            # Convert to edge index format
            edge_indices = np.where(adj_matrix > 0)
            edge_index = torch.tensor(np.vstack(edge_indices), dtype=torch.long)
            edge_weight = torch.tensor(adj_matrix[edge_indices], dtype=torch.float)
            edge_time = torch.tensor(time_matrix[edge_indices], dtype=torch.float)

            # Node features with temporal information
            node_features = torch.tensor(window_data, dtype=torch.float)

            # Add temporal features
            relative_times = torch.tensor(
                (window_timestamps - window_timestamps[0]).reshape(-1, 1),
                dtype=torch.float
            )
            node_features = torch.cat([node_features, relative_times], dim=1)

            # Graph label
            graph_label = torch.tensor(window_labels[-1], dtype=torch.long)

            # Create temporal graph data object
            graph = Data(
                x=node_features,
                edge_index=edge_index,
                edge_attr=torch.cat([edge_weight.unsqueeze(1), edge_time.unsqueeze(1)], dim=1),
                y=graph_label
            )

            graphs.append(graph)

        return graphs

In [15]:
class PositionalEncoding(nn.Module):
    """Fixed Positional encoding for graph input (2D-safe)"""
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() *
                             (-math.log(10000.0) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        self.register_buffer('pe', pe)

    def forward(self, x):
        # x is [num_nodes, feature_dim]
        pe = self.pe[:x.size(0), :]
        return x + pe


class TemporalGraphAttentionLayer(nn.Module):
    """Custom Temporal Graph Attention Layer"""
    def __init__(self, in_dim, out_dim, num_heads=8, dropout=0.1):
        super(TemporalGraphAttentionLayer, self).__init__()

        self.num_heads = num_heads
        self.out_dim = out_dim
        self.head_dim = out_dim // num_heads

        assert out_dim % num_heads == 0

        # Attention mechanisms
        self.spatial_attention = GATConv(in_dim, out_dim, heads=num_heads,
                                       dropout=dropout, concat=False)
        self.temporal_attention = nn.MultiheadAttention(out_dim, num_heads,
                                                       dropout=dropout, batch_first=True)

        # Temporal encoding
        self.temporal_encoder = nn.Linear(2, out_dim)  # edge_weight + edge_time

        # Layer normalization and dropout
        self.norm1 = nn.LayerNorm(out_dim)
        self.norm2 = nn.LayerNorm(out_dim)
        self.dropout = nn.Dropout(dropout)

        # Feed forward network
        self.ffn = nn.Sequential(
            nn.Linear(out_dim, out_dim * 4),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(out_dim * 4, out_dim)
        )

    def forward(self, x, edge_index, edge_attr):
        # Spatial attention
        spatial_out = self.spatial_attention(x, edge_index)
        spatial_out = self.dropout(spatial_out)

        # Residual connection and normalization
        x = self.norm1(x + spatial_out)

        # Temporal attention (treat nodes as sequence)
        temporal_input = x.unsqueeze(0)  # Add batch dimension
        temporal_out, _ = self.temporal_attention(temporal_input, temporal_input, temporal_input)
        temporal_out = temporal_out.squeeze(0)  # Remove batch dimension
        temporal_out = self.dropout(temporal_out)

        # Residual connection and normalization
        x = self.norm2(x + temporal_out)

        # Feed forward network
        ffn_out = self.ffn(x)
        ffn_out = self.dropout(ffn_out)

        # Final residual connection
        return x + ffn_out

class AdvancedTGAT(nn.Module):
    """Advanced Temporal Graph Attention Network for CAN IDS"""
    def __init__(self, input_dim, hidden_dim=256, num_classes=4, num_heads=8,
                 num_layers=4, dropout=0.2):
        super(AdvancedTGAT, self).__init__()

        self.input_projection = nn.Linear(input_dim, hidden_dim)
        self.positional_encoding = PositionalEncoding(hidden_dim)

        # Multiple TGAT layers
        self.tgat_layers = nn.ModuleList([
            TemporalGraphAttentionLayer(hidden_dim, hidden_dim, num_heads, dropout)
            for _ in range(num_layers)
        ])

        # Batch normalization layers
        self.batch_norms = nn.ModuleList([
            BatchNorm(hidden_dim) for _ in range(num_layers)
        ])

        # Global attention pooling
        self.global_attention = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.Tanh(),
            nn.Linear(hidden_dim // 2, 1)
        )

        # Classification head with multiple layers
        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.BatchNorm1d(hidden_dim // 2),
            nn.Linear(hidden_dim // 2, hidden_dim // 4),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.BatchNorm1d(hidden_dim // 4),
            nn.Linear(hidden_dim // 4, num_classes)
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, x, edge_index, edge_attr, batch):
        # Input projection
        x = self.input_projection(x)
        x = self.dropout(x)

        # Apply positional encoding
        x = self.positional_encoding(x)

        # Apply TGAT layers with residual connections
        for i, (tgat_layer, bn) in enumerate(zip(self.tgat_layers, self.batch_norms)):
            residual = x
            x = tgat_layer(x, edge_index, edge_attr)
            x = bn(x)

            # Residual connection (with projection if needed)
            if i > 0:
                x = x + residual

            x = F.relu(x)
            x = self.dropout(x)

        # Global attention pooling
        attention_weights = self.global_attention(x)
        attention_weights = torch.softmax(attention_weights, dim=0)

        # Apply attention weights to pooling
        pooled_features = []
        for batch_id in torch.unique(batch):
            mask = (batch == batch_id)
            batch_features = x[mask]
            batch_weights = attention_weights[mask]

            # Weighted average pooling
            weighted_features = torch.sum(batch_features * batch_weights, dim=0)
            pooled_features.append(weighted_features)

        x_pooled = torch.stack(pooled_features)

        # Classification
        out = self.classifier(x_pooled)

        return out

class CANIDSTrainer:
    def __init__(self, model, device):
        self.model = model.to(device)
        self.device = device
        self.optimizer = torch.optim.Adam(
        model.parameters(), lr=0.0005, weight_decay=1e-5
        )

        self.scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            self.optimizer, mode='max', factor=0.7, patience=5, verbose=True
        )
        self.criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

    def train_epoch(self, train_loader):
        self.model.train()
        total_loss = 0
        correct = 0
        total = 0
        print("🔁 Entered train_epoch()")
        for batch in train_loader:

            batch = batch.to(self.device)

            self.optimizer.zero_grad()
            out = self.model(batch.x, batch.edge_index, batch.edge_attr, batch.batch)
            loss = self.criterion(out, batch.y)

            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
            self.optimizer.step()

            total_loss += loss.item()
            _, predicted = torch.max(out.data, 1)
            total += batch.y.size(0)
            correct += (predicted == batch.y).sum().item()

        return total_loss / len(train_loader), 100 * correct / total

    def evaluate(self, test_loader):
        self.model.eval()
        correct = 0
        total = 0
        all_preds = []
        all_labels = []

        with torch.no_grad():
            for batch in test_loader:
                batch = batch.to(self.device)
                out = self.model(batch.x, batch.edge_index, batch.edge_attr, batch.batch)
                _, predicted = torch.max(out.data, 1)

                total += batch.y.size(0)
                correct += (predicted == batch.y).sum().item()

                all_preds.extend(predicted.cpu().tolist())
                all_labels.extend(batch.y.cpu().tolist())

        accuracy = 100 * correct / total
        return accuracy, all_preds, all_labels

    def train_model(self, train_loader, val_loader, epochs=150):
        print(f"{'Epoch':<10}{'Train Loss':<15}{'Train Acc':<15}{'Val Acc':<15}")
        print("=" * 55)
        best_val_acc = 0
        patience_counter = 0

        for epoch in range(epochs):
            train_loss, train_acc = self.train_epoch(train_loader)
            val_acc, _, _ = self.evaluate(val_loader)

            self.scheduler.step(val_acc)

            if val_acc > best_val_acc:
                best_val_acc = val_acc
                torch.save(self.model.state_dict(), 'best_tgat_can_ids_model.pth')
                patience_counter = 0
            else:
                patience_counter += 1

            print(f'Epoch {epoch:03}: Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, Val Acc: {val_acc:.2f}%')


            # Early stopping
            if patience_counter >= 20:
                print(f"Early stopping at epoch {epoch}")
                break

        return best_val_acc

In [6]:
import torch
import numpy as np


In [9]:



def main():
    # Set device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # Check if running in Colab
    try:
        from google.colab import files
        print("Running in Google Colab")
        print("Using sample data for demonstration...")
        use_sample_data = True
    except ImportError:
        use_sample_data = False

    # Define file paths - update these paths to match your dataset location
    file_paths = {
        '/content/drive/MyDrive/data/DoS_attack_dataset.txt': 'dos',
        '/content/drive/MyDrive/data/Fuzzy_attack_dataset.txt': 'fuzzy',
        '/content/drive/MyDrive/data/Impersonation_attack_dataset.txt': 'impersonation',
        '/content/drive/MyDrive/data/Attack_free_dataset.txt': 'normal'
    }

    # Initialize data processor
    processor = CANDataProcessor()

    # Load and preprocess data
    print("Loading OTIDS dataset...")
    try:
        df = processor.load_otids_data(file_paths)
        if df.empty:
            raise FileNotFoundError("Dataset files not found")
    except:
        print("Dataset files not found, using sample data...")
        df = processor.create_sample_data()

    print(f"Total samples: {len(df)}")
    print(f"Dataset shape: {df.shape}")
    print(f"Label distribution:\n{df['label'].value_counts()}")

    # Preprocess data
    print("\nPreprocessing data...")
    X, y, timestamps = processor.preprocess_can_data(df)

    # Create temporal graphs
    print("Creating temporal graph structures...")
    graphs = processor.create_temporal_graph(X, y, timestamps, window_size=20)
    print(f"Created {len(graphs)} temporal graphs")

    torch.save(graphs, '/content/drive/MyDrive/otids_temporal_graphs_loaded.pt')
    print("✅ Graphs saved to Drive as otids_temporal_graphs.pt")


    # Split data
    train_graphs, test_graphs = train_test_split(graphs, test_size=0.2,
                                               stratify=[g.y.item() for g in graphs],
                                               random_state=42)
    train_graphs, val_graphs = train_test_split(train_graphs, test_size=0.2,
                                              stratify=[g.y.item() for g in train_graphs],
                                              random_state=42)

    print(f"Train graphs: {len(train_graphs)}")
    print(f"Validation graphs: {len(val_graphs)}")
    print(f"Test graphs: {len(test_graphs)}")

    # Create data loaders
    train_loader = DataLoader(train_graphs, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_graphs, batch_size=32, shuffle=False)
    test_loader = DataLoader(test_graphs, batch_size=32, shuffle=False)

    # Initialize model
    input_dim = graphs[0].x.shape[1]  # Features + temporal info
    num_classes = len(np.unique(y))

    model = AdvancedTGAT(input_dim=input_dim,
                        hidden_dim=256,
                        num_classes=num_classes,
                        num_heads=8,
                        num_layers=4,
                        dropout=0.1)

    print(f"\nModel parameters: {sum(p.numel() for p in model.parameters())}")
    print(f"Input dimension: {input_dim}")
    print(f"Number of classes: {num_classes}")

    # Initialize trainer
    trainer = CANIDSTrainer(model, device)

    # Train model
    print("\nTraining TGAT model...")
    best_val_acc = trainer.train_model(train_loader, val_loader, epochs=200)

    # Load best model and evaluate on test set
    try:
        model.load_state_dict(torch.load('best_tgat_can_ids_model.pth'))
    except:
        print("Could not load saved model, using current model state")

    test_acc, test_preds, test_labels = trainer.evaluate(test_loader)

    print(f"\n{'='*50}")
    print(f"FINAL TGAT RESULTS")
    print(f"{'='*50}")
    print(f"Best Validation Accuracy: {best_val_acc:.2f}%")
    print(f"Test Accuracy: {test_acc:.2f}%")

    # Detailed evaluation
    print(f"\n{'='*50}")
    print("CLASSIFICATION REPORT")
    print(f"{'='*50}")
    print(classification_report(test_labels, test_preds,
                              target_names=processor.label_encoder.classes_))

    print(f"\n{'='*50}")
    print("CONFUSION MATRIX")
    print(f"{'='*50}")
    print(confusion_matrix(test_labels, test_preds))

    return model, test_acc, graphs, processor

if __name__ == "__main__":
    # Run the main function
    model, accuracy, graphs, processor = main()

    print(f"\n{'='*60}")
    print("TGAT MODEL TRAINING COMPLETED!")
    print(f"Final Test Accuracy: {accuracy:.2f}%")
    if accuracy >= 98.0:
        print("🎉 TARGET ACCURACY OF 98% ACHIEVED WITH TGAT! 🎉")
    else:
        print("📈 To improve TGAT accuracy further, try:")
        print("- Increase number of attention heads")
        print("- Adjust temporal window size")
        print("- Fine-tune attention mechanisms")
        print("- Increase model depth")
    print(f"{'='*60}")

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Save TGAT model
import shutil
shutil.copy('best_tgat_can_ids_model.pth', '/content/drive/MyDrive/best_tgat_can_ids_model.pth')



Using device: cuda
Running in Google Colab
Using sample data for demonstration...
Loading OTIDS dataset...
Processing /content/drive/MyDrive/data/DoS_attack_dataset.txt...
✔ Loaded 656579 rows from /content/drive/MyDrive/data/DoS_attack_dataset.txt
Processing /content/drive/MyDrive/data/Fuzzy_attack_dataset.txt...
✔ Loaded 591990 rows from /content/drive/MyDrive/data/Fuzzy_attack_dataset.txt
Processing /content/drive/MyDrive/data/Impersonation_attack_dataset.txt...
✔ Loaded 995472 rows from /content/drive/MyDrive/data/Impersonation_attack_dataset.txt
Processing /content/drive/MyDrive/data/Attack_free_dataset.txt...
✔ Loaded 2369398 rows from /content/drive/MyDrive/data/Attack_free_dataset.txt
Total samples: 4613439
Dataset shape: (4613439, 12)
Label distribution:
label
normal           2369398
impersonation     995472
dos               656579
fuzzy             591990
Name: count, dtype: int64

Preprocessing data...
Creating temporal graph structures...
Created 461342 temporal graphs
✅ 

RuntimeError: Numpy is not available

In [ ]:
import torch
import numpy as np
import os
from sklearn.model_selection import train_test_split
from torch_geometric.data import DataLoader
from sklearn.metrics import classification_report, confusion_matrix

from google.colab import drive
drive.mount('/content/drive')  # Mount Drive before anything

def main():
    # Set device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # Define file paths - update these paths to match your dataset location
    file_paths = {
        '/content/drive/MyDrive/data/DoS_attack_dataset.txt': 'dos',
        '/content/drive/MyDrive/data/Fuzzy_attack_dataset.txt': 'fuzzy',
        '/content/drive/MyDrive/data/Impersonation_attack_dataset.txt': 'impersonation',
        '/content/drive/MyDrive/data/Attack_free_dataset.txt': 'normal'
    }

    graph_path = '/content/drive/MyDrive/otids_temporal_graphs_loaded.pt'

    # Initialize data processor
    processor = CANDataProcessor()

    # Load or create graphs
    if os.path.exists(graph_path):
        graphs = torch.load(graph_path)
        print(f"✅ Loaded {len(graphs)} temporal graphs from saved file")
    else:
        print("Loading OTIDS dataset...")
        df = processor.load_otids_data(file_paths)
        print(f"Total samples: {len(df)}")
        print(f"Dataset shape: {df.shape}")
        print(f"Label distribution:\n{df['label'].value_counts()}")

        print("\nPreprocessing data...")
        X, y, timestamps = processor.preprocess_can_data(df)

        print("Creating temporal graph structures...")
        graphs = processor.create_temporal_graph(X, y, timestamps, window_size=20)
        print(f"Created {len(graphs)} temporal graphs")
        torch.save(graphs, graph_path)
        print("✅ Graphs saved to Drive")

    # Split data
    train_graphs, test_graphs = train_test_split(graphs, test_size=0.2,
                                                 stratify=[g.y.item() for g in graphs],
                                                 random_state=42)
    train_graphs, val_graphs = train_test_split(train_graphs, test_size=0.2,
                                                stratify=[g.y.item() for g in train_graphs],
                                                random_state=42)

    print(f"Train graphs: {len(train_graphs)}")
    print(f"Validation graphs: {len(val_graphs)}")
    print(f"Test graphs: {len(test_graphs)}")
    # Temporarily reduce dataset size for faster training



    # Data loaders
    train_loader = DataLoader(train_graphs, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_graphs, batch_size=32, shuffle=False)
    test_loader = DataLoader(test_graphs, batch_size=32, shuffle=False)

    # Model setup
    input_dim = graphs[0].x.shape[1]
    num_classes = len(set([g.y.item() for g in graphs]))

    model = AdvancedTGAT(
        input_dim=input_dim,
        hidden_dim=128,
        num_classes=num_classes,
        num_heads=4,
        num_layers=2,
        dropout=0.05
    )

    print(f"\nModel parameters: {sum(p.numel() for p in model.parameters())}")
    print(f"Input dimension: {input_dim}")
    print(f"Number of classes: {num_classes}")

    # Training
    trainer = CANIDSTrainer(model, device)
    print("\nTraining TGAT model...")
    best_val_acc = trainer.train_model(train_loader, val_loader, epochs=200)

    # Save & evaluate
    model.load_state_dict(torch.load('best_tgat_can_ids_model.pth'))
    test_acc, test_preds, test_labels = trainer.evaluate(test_loader)

    print(f"\n{'='*50}")
    print("FINAL TGAT RESULTS")
    print(f"{'='*50}")
    print(f"Best Validation Accuracy: {best_val_acc:.2f}%")
    print(f"Test Accuracy: {test_acc:.2f}%")

    print("\nCLASSIFICATION REPORT")
    from sklearn.preprocessing import LabelEncoder

    print(classification_report(test_labels, test_preds))



    print("\nCONFUSION MATRIX")
    print(confusion_matrix(test_labels, test_preds))

    return model, test_acc, graphs, processor


if __name__ == "__main__":
    model, accuracy, graphs, processor = main()

    print(f"\n{'='*60}")
    print("TGAT MODEL TRAINING COMPLETED!")
    print(f"Final Test Accuracy: {accuracy:.2f}%")
    if accuracy >= 98.0:
        print("🎉 TARGET ACCURACY OF 98% ACHIEVED WITH TGAT! 🎉")
    else:
        print("📈 To improve TGAT accuracy further, try:")
        print("- Increase number of attention heads")
        print("- Adjust temporal window size")
        print("- Fine-tune attention mechanisms")
        print("- Increase model depth")
    print(f"{'='*60}")




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Using device: cuda
✅ Loaded 461342 temporal graphs from saved file
Train graphs: 295258
Validation graphs: 73815
Test graphs: 92269

Model parameters: 7462213
Input dimension: 12
Number of classes: 4

Training TGAT model...
Epoch     Train Loss     Train Acc      Val Acc        
🔁 Entered train_epoch()
Epoch 000: Train Loss: 0.7510, Train Acc: 74.48%, Val Acc: 74.54%
🔁 Entered train_epoch()
Epoch 001: Train Loss: 0.7498, Train Acc: 74.41%, Val Acc: 73.53%
🔁 Entered train_epoch()
Epoch 002: Train Loss: 0.7379, Train Acc: 74.74%, Val Acc: 72.32%
🔁 Entered train_epoch()
Epoch 003: Train Loss: 0.7312, Train Acc: 74.76%, Val Acc: 75.23%
🔁 Entered train_epoch()
Epoch 004: Train Loss: 0.7325, Train Acc: 74.83%, Val Acc: 61.85%
🔁 Entered train_epoch()
Epoch 005: Train Loss: 0.7332, Train Acc: 74.75%, Val Acc: 73.92%
🔁 Entered train_epoch()


In [19]:
import shutil

shutil.copy('best_tgat_can_ids_model.pth', '/content/drive/MyDrive/best_tgat_can_ids_model.pth')
print("✅ TGAT model saved to Drive")

✅ TGAT model saved to Drive
